# COMP 755

Plan for today

1. Review Generative models for classification
2. Naive Bayes with Bernoulli feature distribution
3. Tuning and evaluating models
    * Cross-Validation
    * ROC plots
    

$$
\renewcommand{\xx}{\mathbf{x}}
\renewcommand{\yy}{\mathbf{y}}
\renewcommand{\zz}{\mathbf{z}}
\renewcommand{\vv}{\mathbf{v}}
\renewcommand{\bbeta}{\boldsymbol{\mathbf{\beta}}}
\renewcommand{\mmu}{\boldsymbol{\mathbf{\mu}}}
\renewcommand{\ssigma}{\boldsymbol{\mathbf{\sigma}}}
\renewcommand{\reals}{\mathbb{R}}
\renewcommand{\loglik}{\mathcal{LL}}
\renewcommand{\penloglik}{\mathcal{PLL}}
\renewcommand{\likelihood}{\mathcal{L}}
\renewcommand{\Data}{\textrm{Data}}
\renewcommand{\given}{ \big| }
\renewcommand{\MLE}{\textrm{MLE}}
\renewcommand{\tth}{\textrm{th}}
\renewcommand{\Gaussian}[2]{\mathcal{N}\left(#1,#2\right)}
\renewcommand{\norm}[1]{\left\lVert#1\right\rVert}
\renewcommand{\ones}{\mathbf{1}}
\renewcommand{\diag}[1]{\textrm{diag}\left( #1 \right)}
\renewcommand{\sigmoid}[1]{\sigma\left(#1\right)}
\renewcommand{\myexp}[1]{\exp\left\{#1\right\}}
\renewcommand{\mylog}[1]{\log\left\{#1\right\}}
\renewcommand{\argmax}{\mathop{\textrm{argmax}}}
$$

# Multivariate Gaussian distribution -- dependent case

Suppose we have $n$ standard random variables  (0 mean, unit variance)
$$
\begin{aligned}
z_i \sim& \Gaussian{0}{1},&  i=1,\dots n
\end{aligned}
$$
and we are given a vector $\mmu$ of length $n$ and a full-rank matrix $A$ of size $n \times n$.

Distribution of $\xx = A\zz + \mu$ is
$$
p(\xx) = \left(2\pi\right)^{-\frac{n}{2}}(\det{\Sigma})^{-\frac{1}{2}}\myexp{-\frac{1}{2}(\xx - \mmu)^T\Sigma^{-1}(\xx-\mmu)}
$$
where $\Sigma = AA^T$.

* $\mu$ is **mean** of the Gaussian
* $\Sigma$ is **covariance** matrix


# Maximum likelihood estimates of mean and covariance

Given data $\{\xx_i \in \reals^n|i=1,\dots,T\}$ maximum likelihood estimates (MLE) of mean and covariance are:
$$
\begin{aligned}
\mmu^{\MLE} &= \frac{1}{T}\sum_{i=1}^T \xx_i\\
\Sigma^{\MLE} & = \frac{1}{T}\sum_{i=1}^T \underbrace{\left(\xx_i - \mmu^{\MLE}\right)\left(\xx_i - \mmu^{\MLE}\right)^T}_{\textrm{ a matrix of size $n \times n$}}
\end{aligned}
$$
Dimensionality
* $\mmu^{\MLE}$ is of same dimension as a single data point $n \times 1$.
* $\Sigma^{\MLE}$ is a matrix of size $n \times n$ 

Note that $\xx\xx^T$ and $\xx^T\xx$ are not the same. Former is a matrix, latter is a scalar.


# Generative models for classification 

There are two ways to factorize joint probability of labels and features
$$
p(y,\xx\given\theta) = p(y\given\xx,\theta)p(\xx\given\theta) =  p(\xx\given y,\theta)p(y\given\theta) 
$$

The second one given us a simple process to *GENERATE* data:

1. First select label according $p(y\given\theta)$, say it was $c$
2. Now generate features $p(\xx\given y=c,\theta)$

Once we have such a model we can obtain the conditional probability $p(y\given\xx)$ using Bayes rule
$$
p(y=c\given \xx) = \frac{p(y=c\given\theta)p(\xx\given y=c,\theta)}{\sum_k p(y=k\given\theta)p(\xx\given y=k,\theta)}
$$
and we can predict label for a new feature vector $\xx$ 

# Naive Bayes

$$
\begin{aligned}
p(y = c \given \pi ) &= \pi_k \\
p(\xx \given y=c, \theta) &= \prod_j p(x_j \given y= c,\theta_{j,c})
\end{aligned}
$$
Parameters are 
* $\pi_c$ prior probability that a sample comes from the class $c$
* $\theta_{j,c}$ parameters for the $j^\tth$ feature for class $c$

In general, there are many variants of Naive Bayes. 

You can choose different distributions for $p(x_j \given y = c)$
* Gaussian -- continuous features
* Bernoulli -- binary features
* Binomial -- count of positive outcomes
* Categorical -- discrete features
* Multinomial -- count of particular discrete outcomes

# Bag of Words representation

Review 188_7:
"I'm a **huge** **classic** film **buff**, but am just **getting** in to **silent** **movies**. 
A **lot** of **silent** **films** don't **hold** my **attention**, ..."

Review 196_9:
"... **fans** of the **silent** **era**, with many **cameos**, **adds** to the **overall** **fun** ..."

Converted into a row of word counts

|   Document_id   | #attention | ... | #classic | ... | #fun   | ... | #silent | ... |
| ---------       | ---        | --- | ---      | --- | ---    | --- | ---     | --- |
|    188_7        |  1         | ... | 1        | ... | 0      | ... | 2       | ... | 
|    196_9        |  0         | ... | 0        | ... | 1      | ... | 1       | ... |
| ...             | ...        | ... |  ...     | ... | ...    | ... | ...     | ... |


Features can also be word presence/absence, rather than counts as above. 

These types of representations are called **bag-of-words**. 

In your homework we use bag-of-words representation of movie reviews to predict sentiment.



# Naive Bayes for spam classification

One approach to classifying e-mail spam (1) vs not spam (0) is to construct a Naive Bayes model using a bag-of-words representation.

Feature vector $\xx$ is $W$ long vector of word presence absence in an e-mail 

$$
\begin{aligned}
p(y =1 ) &= \pi  &&\textrm{prior probability that message is spam} \\
p(y = 0 ) &= 1-\pi  &&\textrm{prior probability that message is  not spam} \\
p(x_{j}  = 1 \given y=1) &= \theta_{1,j} && \textrm{ probability that word $j$ appears in a spam e-mail} \\
p(x_{j}  = 0 \given y=1) &= 1-\theta_{1,j} && \\
p(x_{j}  = 1 \given y=0) &= \theta_{0,j} && \textrm{ probability that word $j$ appears in non-spam e-mail} \\
p(x_{j}  = 0 \given y=0) &= 1-\theta_{0,j} && 
\end{aligned}
$$

Q: What is the size of $\pi$?

Q: What is the size of $\theta$?




# Naive Bayes for spam classification 

$$
\begin{aligned}
p(y =1 ) &= \pi  &&\textrm{prior probability that message is spam} \\
p(y =0 ) &= 1-\pi  &&\textrm{prior probability that message is  not spam} \\
p(x_{j}  = 1 \given y=1) &= \theta_{1,j} && \textrm{ probability that word $j$ appears in a spam e-mail} \\
p(x_{j}  = 0 \given y=1) &= 1-\theta_{1,j} && \textrm{ probability that word $j$ is absent in a spam e-mail}\\
p(x_{j}  = 1 \given y=0) &= \theta_{0,j} && \textrm{ probability that word $j$ is absent in non-spam e-mail} \\
p(x_{j}  = 0 \given y=0) &= 1-\theta_{0,j} && 
\end{aligned}
$$

More compactly for math purposes

$$
\begin{aligned}
p(y) &= \pi^y(1-\pi)^{1-y}\\
p(x_{j}\given y) &= \left[\theta_{1,j}^{x_{j}}(1-\theta_{1,j})^{1-x_{j}}\right]^{y}\left[\theta_{0,j}^{x_{j}}(1-\theta_{0,j})^{1-x_{j}}\right]^{1-y}
\end{aligned}
$$

# Naive Bayes for spam classification -- likelihood

$$
\begin{aligned}
\loglik(\theta,\pi\given X,\yy)=& \underbrace{\sum_{i=1}^N}_{\textrm{samples}} \left[ y_i\log \pi 
+ (1-y_i)\log (1-\pi) \right] \\
+& \underbrace{\sum_{i=1}^N}_{\textrm{samples}}
\underbrace{\sum_{j=1}^W}_{\textrm{words}}  
y_i x_{i,j}\log\theta_{1,j} \\
+& \underbrace{\sum_{i=1}^N}_{\textrm{samples}}
\underbrace{\sum_{j=1}^W}_{\textrm{words}}  
y_i (1-x_{i,j})\log(1-\theta_{1,j}) \\
+& \underbrace{\sum_{i=1}^N}_{\textrm{samples}} 
\underbrace{\sum_{j=1}^W}_{\textrm{words}}  
(1-y_i) x_{i,j}\log(\theta_{0,j}) \\
+& \underbrace{\sum_{i=1}^N}_{\textrm{samples}}
\underbrace{\sum_{j=1}^W}_{\textrm{words}}  
(1-y_i) (1-x_{i,j})\log(1-\theta_{0,j}) 
\end{aligned}
$$


Work out derivatives and updates for $\pi_1$ and $\theta_1,j$ on the board


# Naive Bayes for spam classification -- learning

Given a training data with $N$ labeled e-mails, training a Naive Bayes spam model is accomplished using
$$
\begin{aligned}
\pi_1 &= \frac{\sum_{i=1}^N [y_i = 1]}{N} && \textrm{frequency of spam e-mail in the training data}\\
\theta_{1,j} &=  \frac{\sum_{i=1}^N x_{i,j}*[y_i=1]}{\sum_{i=1}^N [y_i=1]} && \textrm{freqency of word $j$ in spam e-mail}\\
\theta_{0,j} &=  \frac{\sum_{i=1}^N x_{i,j}*[y_i=0]}{\sum_{i=1}^N [y_i=0]} && \textrm{frequency of word $j$ in non-spam e-mail}\\
\end{aligned}
$$



# Naive Bayes for spam classification -- prediction
Prediction for a new e-mail given its bag-of-words representation $\xx$
$$
\begin{aligned}
p(y = 1 \given \xx) &= \frac{p(y=1, \xx)}{p(\xx)} = \frac{p(y=1, \xx)}{p(y=1,\xx) + p(y=0,\xx)}
\end{aligned}
$$
If interested only in the most likely label of a message represented by $\xx$
$$
\argmax_c \log p(y=c, \xx) = \argmax_c \log\pi_c + \sum_{j=1}^W  \left[ x_{j}\log\theta_{c,j} + (1-x_{j})\log(1 - \theta_{c,j}) \right]
$$



# Spam filtering -- smoothing

$$
\begin{aligned}
\pi_1 &= \frac{\sum_{i=1}^N [y_i = 1]}{N} && \textrm{frequency of spam e-mail in the training data}\\
\theta_{1,j} &=  \frac{\sum_{i=1}^N x_{i,j}*[y_i=1]}{\sum_{i=1}^N  [y_i = 1]} && \textrm{freqency of word $j$ in spam e-mail}\\
\theta_{0,j} &=  \frac{\sum_{i=1}^N x_{i,j}*[y_i=0]}{\sum_{i=1}^N [y_i = 0]} && \textrm{frequency of word $j$ in non-spam e-mail}\\
\end{aligned}
$$

Q: What is the value of $\theta_{1,j}$ if the word has never been seen in a spam e-mail in the dataset? Is this a problem? If so, why and how would you fix it?

http://spamassassin.apache.org/

Uses Naive Bayes approach with smoothing -- even though a word has not been seen it does not have probability 0.




# Hyperparameters

In penalized regression we added ridge term
$$
\loglik(\beta) - \frac{\lambda}{2} \sum_{j>0}\beta_j^2
$$

Q: How do we choose $\lambda$? What happens if we try to maximize penalized log-likelihood with respect to $\lambda$?


Q: How do we choose which words to count in our document classifiers? All of them? Do we have enough samples?




# Hyperparameters

Hyperparameters, unlike parameters, typically constrain the model complexity to avoid overfitting.

Overfitting occurs when our performance on training set is optimistic compared to performance on test set

![](tradeoff.png)

# How do we select hyperparameters?

One approach is to split the dataset into three parts:
1. Training
2. Validation
3. Test

Training subset is used to train models for different settings of hyperparameters
for example $\lambda \in \{0.1,0.001,0.0001..\}$

Validation subset is used to evaluate accuracy of the different models we trained and
select the best model.

Test set is used to compute the accuracy of the model selected on validation set.

Q: Why can't we just report accuracy of the best model on the validation set? 

# How do we select hyperparameters if the dataset is small?

Splitting data into three parts makes sense on a large dataset, but can hurt your performance on a small dataset.

We can't look at the test set during training, but validation set seems to be just sitting around

Q: Can we somehow use more of the training and validation data for training? What is the problem of leaving just one sample for validation?

# The idea: cross-validation

First take out test data, then split the remaining data into $k$ parts and treat each part as validation while training on the rest.

An example of $4$-fold cross validation:

0. Split data into disjoint subsets $\Data = \Data_1 \cup \Data_2 \cup \Data_3 \cup \Data_4$
1. Train on $\Data_2 \cup\Data_3\cup\Data_4$ compute $\textrm{Error}_1$ on $\Data_1$ 
2. Train on $\Data_1\cup\Data_3\cup\Data_4$ compute $\textrm{Error}_2$ on $\Data_2$ 
3. Train on $\Data_1\cup\Data_2\cup\Data_4$ compute $\textrm{Error}_3$ on $\Data_3$ 
4. Train on $\Data_1\cup\Data_2\cup\Data_3$ compute $\textrm{Error}_4$ on $\Data_4$ 

![](cv.png)

Report 
$$
\textrm{CVError} = \frac{\textrm{Error}_1 + \textrm{Error}_2 + \textrm{Error}_3 + \textrm{Error}_4}{4}
$$


# k-fold cross-validation for linear regression

![](cvlinreg.png)

# Classification performance -- confusion matrix

Say we trained a classifier and want to see how well it works.

We can report a single number, accuracy, that tells us how often it is right.

However, this hides information about where the classifier fails.

Confusion matrix is given by:

|Predicted \ True     | $y=1$ | $y=0$   |
| ------ | ------ | ------ |
| $\hat{y}=1$ | True Positive  | False Positive |
| $\hat{y}=0$ | False Negative | True Positive |




# Classification performance -- prediction rates

Prediction rates
* true positive rate $$ TPR = \frac{TP}{TP + FN} $$
* false positive rate $$ FPR = \frac{FP}{TN + FP} $$
* true negative rate $$ TNR = \frac{TN}{TN + FP} $$
* false negative rate $$ FNR = \frac{FN}{TP + FN} $$

Note that $TP + FN$ is total number of positive examples. 

Similarly $TN + FP$ is total number of negative examples.

# Classification performance -- ROC curves

Predictions are based on a cutoff
$$
p(y=1|\xx)>\tau
$$
where $\tau$ is typically 0.5.

This particular cutoff will result in a specific prediction rates.

However, you may prefer to tradeoff false positives for false negatives -- health industry does.



# Classification performance -- ROC curves

![](roc.png)

# Today
* Naive Bayes example
* Hyperaparameter selection
* K-fold Cross-validation 
* Prediction rates, confusion matrix and ROC
